In [1]:
import glob
import pickle
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [2]:
#**IMPORTING DATA**
files=glob.glob('../../data/price_index_data/mod/*')

#getting the prices history from each index
data=[]
for file in files:
    data.append(pd.read_csv(file))

#converting the 'date' columns to datetime type
for i in data:
    i['date']=pd.to_datetime(i['date'])

#combining the data together
df=data[0]
for i in data[1:]:
    df=df.merge(i,on='date')

#removing the date column and setting it as the index
df.index=df['date']
df.drop(columns=['date'],inplace=True)

#removing unecessary columns
cols=[i for i in list(df.columns) if 'Unnamed' not in i]
df=df[cols]
df=df.dropna()

In [3]:
cols=[i for i in df.columns if 'close' in i]
df=df[cols]

In [4]:
data={}
targets=['bit_close','eth_close','ada_close','bnb_close']
for target in targets:
    x=df.drop(columns=target)
    y=df[target]**(-0.5)
    temp=pd.DataFrame()
    for col in x.columns:
        for i in range(2,10):
            temp[f'{col}**{i}']=x[col]**i
    temp[target]=y
    temp1=pd.DataFrame(temp.corr()[target])
    temp1=temp1.sort_values(by=target,ascending=False)
    temp1=temp1[(temp1[target]>=0.5) & (temp1[target]<1)]
    res=[]
    for cols in x.columns:
        ind=0
        while ind<len(temp1.index):
            if str(cols) in list(temp1.index)[ind]:
                res.append(list(temp1.index)[ind])
                cols=len(temp1.index)
            else:
                ind+=1
    temp=temp[res]
    temp=pd.DataFrame(temp, columns=temp.columns)
    x=x.merge(temp,on=x.index)
    x.drop(columns='key_0',inplace=True)
    x=x.dropna()
    data[target]=x
for i,j in data.items():       
    for k in range(1,11):
        j['day_'+str(k)]=df[i].shift(-k).values
    data[i]=j.dropna()   

with open('../../data/train/inear_log(price_data).pkl', 'wb') as f:
    pickle.dump(data,f)